# Assignment 3 - Extra : CIFAR-10

## Import Data

In [ ]:
! pip install ipython-autotime

%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=7103e0feeb543897334736ca003386b60da361124a3510d37d9e50d89a05d274
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [ ]:
import numpy as np
import pickle
import tensorflow as tf

time: 1.71 s


In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step
time: 4.43 s


In [ ]:
X_train.shape

(50000, 32, 32, 3)

time: 4.12 ms


## Preprocessing

In [ ]:
ImgDatagenProperty = {
    'rescale' : 1/255,
    'rotation_range' : 40,
    'width_shift_range' : 0.3,
    'height_shift_range' : 0.3,
    'shear_range' : 0.3,
    'zoom_range' : 0.3,
    'horizontal_flip' : True,
    'fill_mode' : 'nearest'
}

time: 4.88 ms


In [ ]:
ImgValDatagenProperty = {
    'rescale' : 1/255,
}

time: 2.05 ms


In [ ]:
ImgModelProperty = {
    'img_height' : 32,
    'img_width' : 32,
    'num_class' : 10,
    'epoch' : 50,
    'batch_size' : 128,
}

time: 2.99 ms


In [ ]:
import tensorflow as tf

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**ImgDatagenProperty).flow(X_train, y_train, batch_size=ImgModelProperty['batch_size'], seed=5555, shuffle=True)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**ImgValDatagenProperty).flow(X_test, y_test, batch_size=ImgModelProperty['batch_size'], seed=5555, shuffle=True)

time: 235 ms


## Model

https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3/preprocess_input

In [ ]:
import tensorflow as tf

def BuildModel(input_dim = (32, 32, 3), n_class = 10) -> tf.keras.Sequential :
    model = tf.keras.Sequential()
    model.add(tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=input_dim))
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dense(128, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(32, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dense(10, activation=tf.keras.activations.softmax))
    # padding_input = tf.keras.layers.ZeroPadding2D(padding=(96, 96), input_shape=input_dim)
    # pretrained = tf.keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet')(padding_input)

    # model = tf.keras.Model(padding_input.input, pretrained)
    return model    

time: 14.8 ms


In [ ]:
model = BuildModel()

58892288/58889256 [==============================] - 1s 0us/step
time: 8.52 s


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 14,784,810
Trainable params: 14,784,810
Non-trainable params: 0
____________________________________________

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

time: 24.4 ms


In [ ]:
import tensorflow as tf

red = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='auto', min_lr=1e-5)

time: 2.69 ms


In [ ]:
%load_ext tensorboard
! rm -rf ./logs/ 

time: 1.83 s


In [ ]:
import datetime
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tsb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

time: 199 ms


In [ ]:
model.fit(
    train_datagen, 
    epochs=ImgModelProperty['epoch'],
    validation_data=val_datagen,
    callbacks=[red, tsb]
)

Epoch 1/50
391/391 [==============================] - 71s 182ms/step - loss: 2.0360 - sparse_categorical_accuracy: 0.1903 - val_loss: 1.9038 - val_sparse_categorical_accuracy: 0.2113 - lr: 0.0010
Epoch 2/50
391/391 [==============================] - 68s 174ms/step - loss: 1.7859 - sparse_categorical_accuracy: 0.2903 - val_loss: 1.5477 - val_sparse_categorical_accuracy: 0.3919 - lr: 0.0010
Epoch 3/50
391/391 [==============================] - 68s 174ms/step - loss: 1.5754 - sparse_categorical_accuracy: 0.3869 - val_loss: 1.4103 - val_sparse_categorical_accuracy: 0.4543 - lr: 0.0010
Epoch 4/50
391/391 [==============================] - 68s 174ms/step - loss: 1.4281 - sparse_categorical_accuracy: 0.4625 - val_loss: 1.3736 - val_sparse_categorical_accuracy: 0.4926 - lr: 0.0010
Epoch 5/50
391/391 [==============================] - 68s 174ms/step - loss: 1.3148 - sparse_categorical_accuracy: 0.5219 - val_loss: 1.1316 - val_sparse_categorical_accuracy: 0.5991 - lr: 0.0010
Epoch 6/50
391/391 [

time: 57min 24s


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
# tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(224, 224, 3)).summary()

time: 1.02 ms
